# SQLAlchemy example
pandas.to_sql removes primary keys in your table, and then the orm can't read your tables properly later. If you don't care about that, then to_sql works for you.

In [3]:
import pandas as pd
#using pd.to_sql removes any primary keys in the table, so we won't use it here
from sqlalchemy import create_engine, Column, Integer, String, Boolean, text, inspect
from sqlalchemy.orm import declarative_base, sessionmaker


In [4]:
#database set up
database_file = 'places.db'
engine = create_engine('sqlite:///'+database_file)
Base = declarative_base() #this needs to be before you declare your class

In [5]:
#input data
data = {
    'id': [1, 2, 3, 4],
    'place_name': ['Parthenon', 'Jupiter', 'Lake Nemi', 'Athens'],
    'place_address': ['123 Main St', '456 Oak Ave', '789 Elm Rd', '101 Pine Blvd'],
    'protected': [True, False, True, False]
}

df = pd.DataFrame(data)

## TimeStamp note
sqlite doesn't support timestamp data types in the database. If you're using dates, save them as string. If you're wanting audit columns, use the following:
```
from datetime import datetime
datetime.now().strftime('%Y-%m-%d %H:%M:%S')
```

In [6]:
class Places(Base):
    __tablename__ = 'places'
    #to get the string lengths, you have to look at the data for the max lengths of the input data
    id = Column(Integer, primary_key=True)
    PlaceName = Column(String(9), unique=True, nullable=False)
    PlaceAddress  = Column(String(13), unique=True, nullable=False)
    Protected  = Column(Boolean,  nullable=True)
#The following line creates table if it doesn't exist,
#   and needs to be after you define your model (your table), so the metadata is properly saved
Base.metadata.create_all(engine)

#sqlalchemy orm uses sessions
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
print("primary key is id :", inspect(Places).primary_key[0].name)
inspector = inspect(engine)
existing_tables = inspector.get_table_names()
print("this says places table exists :", existing_tables)
print("table is also in metadata :", Base.metadata.tables.keys())
#if Base.metadata.create_all(engine) isn't after your class for the data,
#   inspect can see your table, but the metadata can't.

primary key is id : id
this says places table exists : ['places']
table is also in metadata : dict_keys(['places'])


In [8]:
for idx, row in df.iterrows(): #itertuples is faster than iterrows
    # Create a Place object for each row
    print(row)
    place_object = Places(
        PlaceName=row['place_name'],
        PlaceAddress=row['place_address'],
        Protected=row['protected'],
    )
    session.add(place_object)
session.commit()

id                         1
place_name         Parthenon
place_address    123 Main St
protected               True
Name: 0, dtype: object
id                         2
place_name           Jupiter
place_address    456 Oak Ave
protected              False
Name: 1, dtype: object
id                        3
place_name        Lake Nemi
place_address    789 Elm Rd
protected              True
Name: 2, dtype: object
id                           4
place_name              Athens
place_address    101 Pine Blvd
protected                False
Name: 3, dtype: object


In [9]:
all_places = session.query(Places).all() #removing .all() will provide the same result, but use .all()
print("all places:")
for p in all_places:
    print(p.id, p.PlaceName, p.PlaceAddress, p.Protected)

all places:
1 Parthenon 123 Main St True
2 Jupiter 456 Oak Ave False
3 Lake Nemi 789 Elm Rd True
4 Athens 101 Pine Blvd False


In [10]:
all_places = session.execute(text("select * from places"))
print("all places:")
for p in all_places:
    print(p[1]) #one element
    print(p) #whole tuple

all places:
Parthenon
(1, 'Parthenon', '123 Main St', 1)
Jupiter
(2, 'Jupiter', '456 Oak Ave', 0)
Lake Nemi
(3, 'Lake Nemi', '789 Elm Rd', 1)
Athens
(4, 'Athens', '101 Pine Blvd', 0)


In [11]:
places = session.query(Places).filter_by(PlaceName='Parthenon').all()
for p in places:
  print(p.id, p.PlaceName, p.PlaceAddress, p.Protected)

1 Parthenon 123 Main St True


In [12]:
session.close()

In [27]:
# #example of reading in an existing database with the orm
# #disconnect and delete the runtime and then run the following
# #google colab comment and uncomment shortcut - ctrl+/
# #      so highlight all this and ctrl+/ then run
# #uncomment and run the following - the database must exist for this to work
# from sqlalchemy import create_engine, Column, Integer, String, Boolean, text, inspect
# from sqlalchemy.orm import declarative_base, sessionmaker
# from sqlalchemy.ext.automap import automap_base #table must have a primary key defined

# #database set up
# database_file = 'places.db'
# engine = create_engine('sqlite:///'+database_file)
# Base = automap_base()
# Base.prepare(autoload_with=engine)
# print("primary key is id :", inspect(Places).primary_key[0].name)
# inspector = inspect(engine)
# existing_tables = inspector.get_table_names()
# print("this says places table exists :", existing_tables)
# print("table is also in metadata :", Base.metadata.tables.keys())
# Session = sessionmaker(bind=engine)
# session = Session()
# print()
# res = session.query(Places).all()
# for r in res:
#     print(r.id, r.PlaceAddress, r.PlaceName, r.Protected)

primary key is id : id
this says places table exists : ['places']
table is also in metadata : dict_keys(['places'])

1 123 Main St Parthenon True
2 456 Oak Ave Jupiter False
3 789 Elm Rd Lake Nemi True
4 101 Pine Blvd Athens False
